In [2]:
import grafiti as gf
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import squidpy as sq
from sklearn import metrics
import seaborn as sns
import pandas as pd
import anndata as ad
import torch

import warnings
warnings.filterwarnings('ignore')

datadir = "/data1/shahs3/users/mezallj1/data/xenium"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
cells_003 = pd.read_csv(f'{datadir}/raw/003_prepost.obs.celltypesnamed.csv.gz')
cells_003.columns = ['cell_labels','cluster_names','cluster_names_aggtumor']
cells_028 = pd.read_csv(f'{datadir}/raw/028_prepost.obs.celltypesnamed.csv.gz')
cells_028.columns = ['cell_labels','cluster_names','cluster_names_aggtumor']

In [15]:
adata = sc.read_h5ad(f'{datadir}/raw/spectrum_xenium_normalized.h5ad')

In [16]:
adata

AnnData object with n_obs × n_vars = 1419439 × 477
    obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_20_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_150_genes', 'n_counts', 'Sample_ID', 'Patient_ID', 'Treatment', 'Region', 'Site'
    obsm: 'X_pca', 'spatial'
    layers: 'log_norm'

In [52]:
(adata.obs['Patient_ID'].index[adata.obs['Patient_ID']=='SPECTRUM-OV-003']==[i.split('_')[0] for i in cells_003['cell_labels'].values]).any()

True

In [53]:
cells_003

,cell_labels,cluster_names,cluster_names_aggtumor
0,aaaafgco-1_0,Tumor_6,Tumor
1,aaacpopp-1_0,Macrophage_1,Macrophage_1
2,aaaddcac-1_0,Tumor_2,Tumor
3,aaadejcm-1_0,Tumor_1,Tumor
4,aaadelaj-1_0,Endothelial_2,Endothelial_2
...,...,...,...
1051908,oimdpdko-1_1,Plasma_cell,Plasma_cell
1051909,oimecfkl-1_1,T_Naive,T_Naive
1051910,oimefeel-1_1,CD4_T_cell,CD4_T_cell
1051911,oimegapl-1_1,CD4_T_cell,CD4_T_cell


In [48]:
[i.strip('_0') for i in cells_003['cell_labels'].values]

['oilofpaa-1_1',
 'oilohmfn-1_1',
 'oiloicfn-1_1',
 'oiloilea-1_1',
 'oilolpna-1_1',
 'oilomgli-1_1',
 'oilomooc-1_1',
 'oilonbje-1_1',
 'oilpalhh-1_1',
 'oilpamak-1_1',
 'oilpbdnl-1_1',
 'oilpbihc-1_1',
 'oilpbjnm-1_1',
 'oilpciii-1_1',
 'oilpckhh-1_1',
 'oilpefmp-1_1',
 'oilpegab-1_1',
 'oilpfnco-1_1',
 'oilphdhc-1_1',
 'oilphhjn-1_1',
 'oilpjenk-1_1',
 'oilpjgog-1_1',
 'oilplcnl-1_1',
 'oilplglc-1_1',
 'oilplhlb-1_1',
 'oilplmpm-1_1',
 'oilpngfk-1_1',
 'oilpogpj-1_1',
 'oilpolcj-1_1',
 'oilppbli-1_1',
 'oilppekb-1_1',
 'oimaadmn-1_1',
 'oimaelmp-1_1',
 'oimagfbd-1_1',
 'oimagjch-1_1',
 'oimagkig-1_1',
 'oimaiimo-1_1',
 'oimajejf-1_1',
 'oimakcpa-1_1',
 'oimakijn-1_1',
 'oimaleka-1_1',
 'oimamaof-1_1',
 'oimamnmf-1_1',
 'oimanneo-1_1',
 'oimapaci-1_1',
 'oimbalka-1_1',
 'oimbbjjc-1_1',
 'oimbbmpi-1_1',
 'oimbbofp-1_1',
 'oimbcjjg-1_1',
 'oimbdnek-1_1',
 'oimbgngn-1_1',
 'oimbgnlo-1_1',
 'oimbignk-1_1',
 'oimbilkd-1_1',
 'oimbkloi-1_1',
 'oimbklph-1_1',
 'oimbmdff-1_1',
 'oimbmpdd-1_1

In [39]:
adata.obs['Patient_ID'].index[adata.obs['Patient_ID']=='SPECTRUM-OV-003'][1000]

'addifjkb-1'

In [40]:
cells_003['cell_labels'][1000]

'addifjkb-1_0'

In [32]:
adata

AnnData object with n_obs × n_vars = 1419439 × 477
    obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_20_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_150_genes', 'n_counts', 'Sample_ID', 'Patient_ID', 'Treatment', 'Region', 'Site'
    obsm: 'X_pca', 'spatial'
    layers: 'log_norm'